In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plotGraph(resultsFile, mode):
    
    # Load data
    dtw_df = resultsFile
    unsync_df = pd.read_csv('unsync_report.csv')
    
    # Calculate expected offsets
    unsync_df['expected_offset'] = unsync_df['right_arm'] - unsync_df['left_arm']
    
    # Constants
    tolerance = 50
    dtw_threshold = 2
    
    # Prepare data
    box_data = []           # For boxplot values
    box_positions = []      # X-axis positions of boxplots
    bar_positions = []      # X-axis positions of expected offset bars
    bar_heights = []        # Heights of expected offset bars
    xtick_labels = []       # Custom tick labels
    current_pos = 0         # Position tracker
    
    for _, row in unsync_df.iterrows():
        subject = row['subject']
        expected_offset = row['expected_offset']
        
        # Filter DTW data for this subject
        subject_df = dtw_df.copy()
        if subject_df.empty:
            print(f"Skipping {subject} (no DTW data)")
            continue
        
        # Calculate observed offset and error
        subject_df['observed_offset'] = subject_df['Ref Start'] - subject_df['Target Start']
        subject_df['error'] = subject_df['observed_offset'] - expected_offset
        
        # Filter best matches
        best_matches = subject_df[
            (subject_df['DTW Distance'] >= 0) & (subject_df['DTW Distance'] <= dtw_threshold) &
            (subject_df['error'].abs() <= tolerance)
        ]
        
        # Append bar and box data
        bar_positions.append(current_pos)
        bar_heights.append(expected_offset)
        
        box_data.append(subject_df['observed_offset'])
        box_positions.append(current_pos + 1)
        
        box_data.append(best_matches['observed_offset'])
        box_positions.append(current_pos + 2)
        
        # Labels for the subject's 3-part group
        xtick_labels.extend([f'{subject}\nExpected', f'{subject}\nAll', f'{subject}\nBest'])
        
        current_pos += 4  # Leave space between subjects
    
    # Plotting
    plt.figure(figsize=(max(12, current_pos * 0.6), 6))
    ax = plt.gca()
    
    # Plot bars (Expected Offsets)
    ax.bar(bar_positions, bar_heights, width=0.6, color='skyblue', label='Expected Offset')
    
    # Plot boxplots
    box = ax.boxplot(box_data, positions=box_positions, widths=0.6, patch_artist=True)
    
    # Color the boxes
    colors = ['lightgreen', 'orange'] * (len(box_data) // 2)
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        # Draw subject-specific tolerance bands (horizontal dashed lines) per subject
    for pos, expected in zip(bar_positions, bar_heights):
        ax.hlines(y=expected + tolerance, xmin=pos - 0.5, xmax=pos + 2.5, colors='green', linestyles='-', linewidth=1)
        ax.hlines(y=expected - tolerance, xmin=pos - 0.5, xmax=pos + 2.5, colors='green', linestyles='-', linewidth=1)
        ax.hlines(y=expected , xmin=pos - 0.5, xmax=pos + 2.5, colors='red', linestyles='--', linewidth=0.8)
    
    # Add 0 line and tolerance band
    ax.axhline(0, color='black', linewidth=1, linestyle='-')
    
    # Labels and layout
    ax.set_xticks([p for group in zip(bar_positions, box_positions[::2], box_positions[1::2]) for p in group])
    ax.set_xticklabels(xtick_labels, rotation=45, ha='right')
    ax.set_ylabel('Offset')
    ax.set_title('Expected vs Observed Offsets for All Subjects')
    ax.grid(True)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(f"graphs/{mode}.png", dpi=300, bbox_inches='tight')
    # plt.show()

if __name__ == "__main__":
    dtw_results = pd.read_csv('dtw_results.csv')
    os.makedirs('graphs', exist_ok=True)

    for mode in ["r2l_a", "l2r_a", "r2l_l", "l2r_l"]:
        filtered_dtw_results = dtw_results[dtw_results['Mode'] == mode]
        plotGraph(filtered_dtw_results, mode)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Load CSVs ---
dtw_df = pd.read_csv('dtw_results.csv')
unsync_df = pd.read_csv('unsync_report.csv')

# Extract numeric subject IDs from 'Subject' columns for sorting & filtering
dtw_df['Subject_Num'] = dtw_df['Subject'].str.extract(r'(\d+)').astype(int)
unsync_df['Subject_Num'] = unsync_df['subject'].str.extract(r'(\d+)').astype(int)

# Map mode to limb type
mode_to_limb = {
    'r2l_a': 'arm',
    'l2r_a': 'arm',
    'r2l_l': 'leg',
    'l2r_l': 'leg'
}

# Function to remove outliers using IQR
def remove_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series[(series >= lower) & (series <= upper)]

# Get all unique modes
modes = dtw_df['Mode'].unique()

for mode in modes:
    limb_type = mode_to_limb.get(mode)
    
    # Get intersection of subjects sorted numerically
    subjects = sorted(set(dtw_df['Subject_Num']).intersection(set(unsync_df['Subject_Num'])))
    
    expected_list = []
    actual_list = []
    subject_labels = []
    
    for subject_num in subjects:
        # Get expected offset from unsync_df
        unsync_row = unsync_df[unsync_df['Subject_Num'] == subject_num]
        if unsync_row.empty:
            continue
        expected_offset = unsync_row[f'unsync_{limb_type}'].values[0]
        
        # Get DTW data for this subject and mode
        subj_mode_df = dtw_df[(dtw_df['Subject_Num'] == subject_num) & (dtw_df['Mode'] == mode)]
        if subj_mode_df.empty:
            continue
        
        # Top 30% DTW distances (smallest)
        top_count = max(1, int(len(subj_mode_df) * 0.3))
        top_df = subj_mode_df.nsmallest(top_count, 'DTW Distance')
        
        # Calculate observed offsets
        top_df = top_df.copy()  # Avoid SettingWithCopyWarning
        top_df['observed_offset'] = top_df['Ref Start'] - top_df['Target Start']
        
        # Remove outliers
        filtered_offsets = remove_outliers_iqr(top_df['observed_offset'])
        
        # Median actual offset (absolute value)
        median_offset = abs(filtered_offsets.median())
        
        # Append to lists
        expected_list.append(expected_offset)
        actual_list.append(median_offset)
        
        # Get original subject label
        label = dtw_df[dtw_df['Subject_Num'] == subject_num]['Subject'].iloc[0]
        subject_labels.append(label)
    
    # --- Plot ---
    x = np.arange(len(subject_labels))
    width = 0.35
    
    plt.figure(figsize=(12,6))
    plt.bar(x - width/2, expected_list, width=width, label='Expected', color='skyblue')
    plt.bar(x + width/2, actual_list, width=width, label='Actual Median (abs)', color='orange')
    
    plt.xticks(x, subject_labels, rotation=45)
    plt.ylabel('Offset')
    plt.title(f'Expected vs Actual Median Offsets (abs) - Mode {mode}')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
